In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pyplot as plt

ROOT_PATH = os.path.normpath(os.path.join(os.getcwd(), os.pardir))
ROOT_PATH

'/home/franchetto4/github/StatMatMethodsAI'

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    data = pd.read_csv("/content/drive/MyDrive/data.csv")
except:
    data = pd.read_csv(os.path.join(ROOT_PATH, "data", "data.csv"))
# Load the data

print("Shape of data {}".format(data.shape))

In [ ]:
print(data.head())

In [ ]:
# Convert into array
data = np.array(data)

# Split into samples and labels (X and Y)
X = data[:, 1:]
X = X.T

Y = data[:, 0]

print(X.shape, Y.shape)

d, N = X.shape

In [ ]:
def get_data_from_index(X, Y, indexes):
    
    # start from empty lists for both samples and labels
    final_X = []
    final_Y = []

    # for each chosen label
    for k in indexes:
        # find which samples have label=k
        idxs_k = (Y == k)
        # slice the samples and append them to a list
        final_X.append( X[:, idxs_k] )
        # same thing to the labels
        final_Y.append( Y[idxs_k] )

    # concatenate together all the previous iterations
    X = np.concatenate(final_X, axis=1)
    Y = np.concatenate(final_Y)

    # return the new dataset and labels
    return X, Y

In [ ]:
indeces = [0, 6, 9]
X, Y = get_data_from_index(X, Y, indeces)
print("X shape: {}, Y shape: {}".format(
    X.shape,
    Y.shape
))

print("Kept samples choosing indeces {}: {:.2f}%".format(
    indeces,
    X.shape[1]/N*100
))

In [ ]:
# 80% to train, the rest to test
train_split = 0.8
N_train = round(X.shape[1]*train_split)

In [ ]:
def data_splits(X, Y, N_train):
    N = X.shape[1]

    # getting an array with indeces from 0 to N-1
    indeces = np.arange(N)
    # shuffling randomly
    np.random.shuffle(indeces)

    # get the first N_train for the train split (but now they are random)
    train_idx = indeces[:N_train]
    # the rest are for test split
    test_idx = indeces[N_train:]

    # slice the original datasets with an index array
    X_train = X[:, train_idx]  
    Y_train = Y[train_idx]
    
    X_test = X[:, test_idx]
    Y_test = Y[test_idx]

    # put in tuples the two splits
    return (X_train, Y_train), (X_test, Y_test)

In [ ]:
# get the train and test splits both for samples and for lables
(X_train, Y_train), (X_test, Y_test) = data_splits(X, Y, N_train)

print("X_train shape: {}, Y_train shape: {}".format(
    X_train.shape,
    Y_train.shape
))

In [ ]:
# set the dimensionality of the reduction
k = 2

In [ ]:
def PCA(X, k):

    # Find the centroid of the dataset
    centroid = np.mean(X, axis=1)

    # Translate the whole dataset so that its center is in 0
    X_c = X - centroid.reshape((d, 1))

    # Compute SVD on the shifted dataset matrix
    U, S, VT = np.linalg.svd(X_c, full_matrices=False)

    # Take only the first k columns on the U matrix: this is now the projection matrix for the PCA
    U_k = U[:, :k]

    # print("Projection matrix shape: {}".format(U_k.shape))

    # Transpose the projection matrix and apply it to the dataset
    return U_k.T @ X

In [ ]:
X_PCA_train = PCA(X_train, k)
print("Projected dataset shape: {}".format(X_PCA_train.shape))

In [ ]:
def LDA(X, Y, k):
    # get unique label values
    unique_idxs = np.unique(Y)
    
    # create the clusters divided by class
    clusters = []
    for i in unique_idxs:
        cluster = X[:, (Y==i)]
        clusters.append(cluster)

    # start with constructing the WITHIN-CLUSTER scatter matrix
    # compute the centroids for each cluster
    centroids = [np.mean(cluster, axis=1) for cluster in clusters]
    # shift each cluster by their centroid so their center is in 0
    shifted_clusters = [cluster - centroid.reshape((d, 1)) 
        for cluster, centroid in zip(clusters, centroids)]
    # concatenate the shifted clusters
    Xw = np.concatenate(shifted_clusters, axis=1)
    # compute the within-cluster scatter matrix (how far is each sample from its centroid, more or less)
    Sw = Xw @ Xw.T

    # second step: construction of the BETWEEN-CLUSTER scatter matrix
    # repeat each centroid as many times as the number of samples in their cluster
    repeated_centroids = [np.repeat(centroid.reshape(d, 1), cluster.shape[1], axis=1)
        for cluster, centroid in zip(clusters, centroids)]
    # concatenate them all
    Xbar = np.concatenate(repeated_centroids, axis=1)

    # find the global centroid of the data
    global_centroid = np.mean(X, axis=1)
    # shift the "repeated centroids matrix" by the global centroid
    Xbarc = Xbar - global_centroid.reshape((d, 1))
    # compute the between-cluster scatter matrix (how far is each centroid from the global one, more or less)
    Sb = Xbarc @ Xbarc.T

    try:
        # if the within-cluster scatter matrix is SPD, compute its cholesky decomposition
        L = np.linalg.cholesky(Sw)
    except:
        # otherwise, add a small perturbation in the form of the identity matrix
        epsilon = 1e-6
        # this shifts the eigenvalues to the right by epsilon
        # REMARK: for any matrix X, X@X^T is SPD (x^T @ A @ x >= 0), so only numerical error can make it non SPD
        # this is why epsilon is enough to bring it back to SPD
        Sw = Sw + epsilon * np.eye(Sw.shape[0])

        # once it is SPD, compute its cholesky decomposition
        L = np.linalg.cholesky(Sw)

    # Compute the first k eigenvector decomposition of L^-1 @ Sb @ L
    _, W = scipy.sparse.linalg.eigs(np.linalg.inv(L) @ Sb @ L, k=k)
    # Sb should be SPD and L^-1 @ Sb @ L is just a change of basis, so its eigenvalues should remain all POSITIVE and REAL
    # but numerical errors can add an imaginary component, so we assume that the latter is small and take only the real component
    W = np.real(W)
    
    # Compute Q, the projection matrix of LDA 
    Q = np.linalg.inv(L).T @ W

    # print("Projection matrix shape: {}".format(Q.shape))

    # Compute the LDA projection on the initial dataset
    return Q.T @ X
    

In [ ]:
X_LDA_train = LDA(X_train, Y_train, k)
print("Projected dataset shape: {}".format(X_LDA_train.shape))

In [ ]:
def plot_clusters_2D(X, Y):
    scatter = plt.scatter(X[0, :], X[1, :], c=Y)
    plt.legend(handles=scatter.legend_elements()[0], labels=[str(y) for y in np.unique(Y)])
    plt.show()

In [ ]:
plot_clusters_2D(X_PCA_train, Y_train)

In [ ]:
plot_clusters_2D(X_LDA_train, Y_train)

In [ ]:
def avg_distance_centroids(X, Y):
    # get unique label values
    unique_idxs = np.unique(Y)
    
    # create the clusters divided by class
    distances = []
    centroids = []
    for i in unique_idxs:
        cluster = X[:, (Y==i)]
        d, N = cluster.shape
        centroid = np.mean(cluster, axis=1)
        print("Coordinates of centroid corresponding to index {}: [{:.2f}, {:.2f}]".format(
            i,
            centroid[0],
            centroid[1]
        ))

        distances.append([np.linalg.norm(cluster[:,j] - centroid) for j in range(N)])
        centroids.append(centroid)

    global_centroid = np.mean(X, axis=1)
    print("Coordinates of global centroid: [{:.2f}, {:.2f}]".format(
        global_centroid[0],
        global_centroid[1]
    ))

    centroid_avg_distance_to_global = np.mean([np.linalg.norm(c-global_centroid) for c in centroids])
    return np.mean(np.concatenate(distances)), centroid_avg_distance_to_global


In [ ]:
distance_PCA, avg_centroid_distance = avg_distance_centroids(X_PCA_train, Y_train)
print("Average distance from point to corresponding centroid with PCA: {:.2f}".format(distance_PCA))
print("After normalization by average centroid distance: {:.2f}".format(distance_PCA / avg_centroid_distance))

In [ ]:
distance_LDA, avg_centroid_distance = avg_distance_centroids(X_LDA_train, Y_train)
print("Average distance from point to corresponding centroid with PCA: {:.2f}".format(distance_LDA))
print("After normalization by average centroid distance: {:.2f}".format(distance_LDA / avg_centroid_distance))

In [ ]:
X_PCA_test = PCA(X_test, k)
distance_PCA_test, avg_centroid_distance = avg_distance_centroids(X_PCA_test, Y_test)
print("Average distance from point to corresponding centroid with PCA: {:.2f}".format(distance_PCA_test))
print("After normalization by average centroid distance: {:.2f}".format(distance_PCA_test / avg_centroid_distance))

In [ ]:
X_LDA_test = LDA(X_test, Y_test, k)
distance_LDA_test, avg_centroid_distance = avg_distance_centroids(X_LDA_test, Y_test)
print("Average distance from point to corresponding centroid with PCA: {:.2f}".format(distance_LDA_test))
print("After normalization by average centroid distance: {:.2f}".format(distance_LDA_test / avg_centroid_distance))

### Explanation of previous results

PCA only projects the initial datapoint on their main components given by the eigenvectors of the reduced $U_k$ matrix.

LDA instead takes into consideration both their intra-cluster covariance and also their between-cluster covariance when computing Sb and Sw, so the datapoints are not only projected on their main components, but also "normalized".

This explains the enormous difference between the PCA average distance of ~500 and the small ~0.01 distance for LDA. When normalize the average distance point-centroid with the average distance centroid-global_centroid, PCA returns ~0.8 while LDA returns ~0.3, meaning that the two algorithms actually behave similarly. However, LDA still produces a lower number, meaning that, on average, the points are nearer to their centroid, fact that is very clear also through the plots.

In [ ]:
def classify_sample(sample, centroids, classes):
    distances = [np.linalg.norm(sample-c) for c in centroids]
    return classes[np.argmin(distances)]

def get_accuracy(X_train, Y_train, X_test, Y_test):
    # get unique label values
    classes = np.unique(Y_train)
    
    centroids = []
    # create the clusters divided by class
    for i in classes:
        cluster = X_train[:, (Y_train==i)]
        centroids.append(np.mean(cluster, axis=1))

    correct_predictions = 0
    for sample, gt_class in zip(X_test.T, Y_test):
        predicted_class = classify_sample(sample, centroids, classes)
        if predicted_class == gt_class:
            correct_predictions += 1

    return correct_predictions / len(Y_test)    

In [ ]:
print("PCA accuracy on test set: {:.2f}%".format(get_accuracy(X_PCA_train, Y_train, X_PCA_test, Y_test)*100))

In [ ]:
print("LDA accuracy on test set: {:.2f}%".format(get_accuracy(X_LDA_train, Y_train, X_LDA_test, Y_test)*100))

In [ ]:
def complete_experiment(data, digits, k, train_split):
    X = data[:, 1:]
    X = X.T
    Y = data[:, 0]
    
    X, Y = get_data_from_index(X, Y, digits)

    N_train = round(X.shape[1]*train_split)

    (X_train, Y_train), (X_test, Y_test) = data_splits(X, Y, N_train)

    X_PCA_train = PCA(X_train, k)
    X_LDA_train = LDA(X_train, Y_train, k)

    X_PCA_test = PCA(X_test, k)
    X_LDA_test = LDA(X_test, Y_test, k)

    acc_PCA = get_accuracy(X_PCA_train, Y_train, X_PCA_test, Y_test)
    acc_LDA = get_accuracy(X_LDA_train, Y_train, X_LDA_test, Y_test)

    return acc_PCA, acc_LDA

In [ ]:
k_values = np.arange(2,103,10)
results = [complete_experiment(data, [0, 6, 9], k, 0.8) for k in k_values]

In [ ]:
k_values = np.arange(2,103,10)

In [ ]:
plt.plot(k_values, [r[0] for r in results], k_values,  [r[1] for r in results])
plt.legend(["PCA", "LDA"])
plt.show()

In [ ]:
digit_values = [
    [0, 6, 9],
    [2, 5],
    [8, 4, 7, 3],
    # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    [1, 7]
]

results = [complete_experiment(data, d, 2, 0.8) for d in digit_values]

In [ ]:
x_range = np.arange(0, len(digit_values))
plt.plot(x_range, [r[0] for r in results], x_range, [r[1] for r in results])
plt.xticks(digit_values)
plt.legend(["PCA", "LDA"])
plt.show()